# 1 Last week we used the activation function. Why is this not used with backpropagation?

__A:__ The function is not differentiable.

# 2 What is the minimum number of hidden neuron layers needed in order to approximate an arbitrary continuous function, and why?

__A:__ One hidden layer is the minimum. "Universal approximation theorem: Any continuous function
can be approximated by a neural network with a single
hidden layer", slide 54 https://www.uio.no/studier/emner/matnat/ifi/INF3490/h18/timeplan/slides/lecture6-1pp.pdf


Without a hidden layer, it is impossible to approximate non-linear separable problems. The first layer gives lines, and the second layer opens up the dimension to two, so e.g. traingles can be made. Furthermore, a compbination of enough traingles can approximate any shape. See slide 12 https://www.uio.no/studier/emner/matnat/ifi/INF3490/h18/timeplan/slides/lecture6-1pp.pdf

<mark> UNCLEAR <br>
    
    
    
# 3 Why do we use a validation set? Describe how the three different cross-validation methods presented in the lecture slides work, and what their advantages and disadvantages are.
We want the model to generalize well, meaning that it works well on other data than the data the model was estimated on. When fitting the model to the full dataset, the model will typically not fit well for other data. In oredr to create a model that generalizes well, the data is split into a training and a validation set. The model is fit to the training set, and is then evaluated on the validation set. The model performance on the validation set describes how well the model generalizes. <br>

Three crossvalidation methods. <br> 
1) Split into training and validation (and possible test). Works as the previous paragraph. <br>
2) K-fold cross-validation. Split into K number of folds. Each fold performs as validation set once, while at the same time the model is fit to the remaining folds. <br>
3) Bootstrapping. Draw random observations with replacement and use the rest as validation. Repeat.

# 4 Implement the MLP shown below, and train it to correctly perform the XOR function.

In [112]:
import numpy as np
class NN:
    """ Multi layer. Single input"""
    
    def __init__(self, nodeNumbers, activationFunction, learningRate, targets, inputs, errorTolerance, test=False):
        self.nodeNumbers, self.activationFunction, self.learningRate, self.targets, self.inputs, self.errorTolerance  \
        = nodeNumbers, activationFunction, learningRate, targets, inputs, errorTolerance
        
        self.weightMatrices = []
        self.weightedInputs = []
        self.outputs = []
        self.deltaVectors = []
        
        
        if not test:
            for nodeNumber in range(len(nodeNumbers)-1):
                self.weightMatrices.append(np.random.random_sample((nodeNumbers[nodeNumber+1], \
                                                                    nodeNumbers[nodeNumber+1] +1)) - .5)
                self.weightedInputs.append(np.zeros(nodeNumbers[nodeNumber+1]))
                self.outputs.append(np.zeros(nodeNumbers[nodeNumber+1]))
                self.deltaVectors.append(np.zeros(nodeNumbers[nodeNumber]+1))
            
        else:
            self.weightMatrices.append(np.array(((1., -1., 0.), (1., 0., 1.))))
            self.weightMatrices.append(np.array(((1., 1., 0.), (1., -1., 1.)))) # Assumed same weights bias nodes both layers
            self.inputs = np.array((1, 0, 1))

        
    def differentiateActivationFunction(self):
        return 1
    
    def forward(self):
        self.weightedInputs.append(self.weightMatrices[0] @ self.inputs)
        self.outputs.append(self.activationFunction(self.weightedInputs[-1]))

        for layer in range(1, len(self.nodeNumbers)-1):
            outputsIncludingBias = np.concatenate([[1], self.outputs[-1]])
            self.weightedInputs.append(self.weightMatrices[layer] @ outputsIncludingBias)
            self.outputs.append(self.activationFunction(self.weightedInputs[-1]))
            
    def calculateErrors(self):
        self.error2 = 0
        for outputNodeNumber in range(len(self.outputs)):
            self.error2 += (self.targets[outputNodeNumber] - \
                           self.outputs[-1][outputNodeNumber])**2
        
    def backward(self):
        deltaValues = np.array([(self.outputs[-1][outputNumber] - self.targets[outputNumber])\
                      *self.differentiateActivationFunction() \
                      for outputNumber in range(len(self.targets))])
        self.deltaVectors.append(deltaValues)
        
        outputsIncludingBias = np.concatenate([[1], self.outputs[-2]])
        for hiddenNodeNumber in range(self.nodeNumbers[-2] + 1):
            for outputNumber in range(len(self.targets)):
                self.weightMatrices[-1][outputNumber, hiddenNodeNumber] -= \
                self.learningRate*self.deltaVectors[0][outputNumber]*outputsIncludingBias[hiddenNodeNumber]
        
        #deltaVec = np.zeros((2,2)) # HERE
        for layerNumber in range(len(self.nodeNumbers)-2):
            for hiddenNodeNumber in range(self.nodeNumbers[-1-(layerNumber+1)] + 1):
                deltaSum = 0
                print('self.deltaVectors', self.deltaVectors)
                for outputNumber in range(len(self.targets)):
                    deltaSum += self.deltaVectors[-1-layerNumber][outputNumber] \
                    * self.weightMatrices[-1][outputNumber,hiddenNodeNumber] # Check order indices
                deltaVectors.insert(0, deltaSum*self.activationFunction(self.weightedInputs[0]))# Avoid hard coding later
                #print('len(deltaVectors[-2-layerNumber])', len(self.deltaVectors[-2-layerNumber]))
        
def activationFunction(x):
    return x
        
numberOfNodes,  activationFunction, learningRate, targets, inputs, errorTolerance = \
[2,2,2], activationFunction, 0.1, [1, 0], [0, 1], 1e-6
test = True
nn=NN(numberOfNodes, activationFunction, learningRate, targets, inputs, test, errorTolerance)
print('nn.weightMatrices',nn.weightMatrices)

nn.forward()
print('nn.weightedInputs',nn.weightedInputs)
print('nn.outputs',nn.outputs)

nn.calculateErrors()
print('nn.error2',nn.error2)

nn.backward()
print('nn.deltaVectors',nn.deltaVectors)
print('weightMatrices', nn.weightMatrices)



nn.weightMatrices [array([[ 1., -1.,  0.],
       [ 1.,  0.,  1.]]), array([[ 1.,  1.,  0.],
       [ 1., -1.,  1.]])]
nn.weightedInputs [array([1., 2.]), array([2., 2.])]
nn.outputs [array([1., 2.]), array([2., 2.])]
nn.error2 5.0
self.deltaVectors [array([1., 2.])]


NameError: name 'deltaVectors' is not defined